In [1]:
import pandas as pd
import yfinance as yf
import requests

In [2]:
start_date = "2023-01-02"
end_date = "2025-12-03"

btc = yf.download("BTC-USD", start=start_date, end=end_date)
eth = yf.download("ETH-USD", start=start_date, end=end_date)

btc.head(), eth.head()

C:\Users\Reshal\AppData\Local\Temp\ipykernel_11080\2380075598.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  btc = yf.download("BTC-USD", start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
C:\Users\Reshal\AppData\Local\Temp\ipykernel_11080\2380075598.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  eth = yf.download("ETH-USD", start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


(Price              Close          High           Low          Open  \
 Ticker           BTC-USD       BTC-USD       BTC-USD       BTC-USD   
 Date                                                                 
 2023-01-02  16688.470703  16759.343750  16572.228516  16625.509766   
 2023-01-03  16679.857422  16760.447266  16622.371094  16688.847656   
 2023-01-04  16863.238281  16964.585938  16667.763672  16680.205078   
 2023-01-05  16836.736328  16884.021484  16790.283203  16863.472656   
 2023-01-06  16951.968750  16991.994141  16716.421875  16836.472656   
 
 Price            Volume  
 Ticker          BTC-USD  
 Date                     
 2023-01-02  12097775227  
 2023-01-03  13903079207  
 2023-01-04  18421743322  
 2023-01-05  13692758566  
 2023-01-06  14413662913  ,
 Price             Close         High          Low         Open      Volume
 Ticker          ETH-USD      ETH-USD      ETH-USD      ETH-USD     ETH-USD
 Date                                                        

In [3]:
btc.to_csv("../data/raw/btc_prices.csv")
eth.to_csv("../data/raw/eth_prices.csv")

In [4]:
import pandas as pd
import requests

# get all available history from Alternative.me API
url = "https://api.alternative.me/fng/?limit=0&format=json"

resp = requests.get(url, timeout=10)
data = resp.json()

# convert to DataFrame (API returns list under 'data')
fg = pd.DataFrame(data["data"])

# convert dtypes
fg["timestamp"] = pd.to_datetime(fg["timestamp"].astype(int), unit="s")
fg["value"] = fg["value"].astype(int)

# set datetime index and sort
fg = fg.set_index("timestamp").sort_index()

fg.head()

,value,value_classification,time_until_update
timestamp,,,
2018-02-01,30,Fear,NaN
2018-02-02,15,Extreme Fear,NaN
2018-02-03,40,Fear,NaN
2018-02-04,24,Extreme Fear,NaN
2018-02-05,11,Extreme Fear,NaN


In [5]:
fg.to_csv("../data/raw/fear_greed_index.csv")
fg.shape

(2861, 3)

In [6]:
start_date = btc.index.min()
end_date = btc.index.max()

fg = fg.loc[start_date:end_date]
fg.to_csv("../data/raw/fear_greed_index_trimmed.csv")
fg.shape

(1065, 3)